# Otimização de Rotas Médicas (VRP com Algoritmo Genético)
Este notebook demonstra, de forma **fiel ao código do projeto**, a execução do Algoritmo Genético para otimização de rotas médicas (VRP) com **população semeada** a partir das rotas reais.

O fluxo geral é:
1. Garantir o ambiente (conda env `fiap_tsp`).
2. Executar o _script_ gerador de dados (`gerar_csv_entregas_rotas_iniciais.py`) – opcional se os CSVs já estiverem gerados.
3. Carregar `veiculos.csv`, `entregas.csv` e `rotas_iniciais.csv`.
4. Executar o GA VRP (`executar_ga_vrp`).
5. Analisar o histórico de fitness.
6. Explorar as rotas resultantes e gerar o mapa com o Folium.


## 1. Imports e configuração geral

In [ ]:
from pathlib import Path
import os
import sys

# Quando roda via nbconvert, o cwd costuma ser .../optimization/notebooks
cwd = Path.cwd().resolve()

# Se estivermos dentro de notebooks/, sobe 1 nível para chegar em optimization/
PROJECT_ROOT = cwd.parent if cwd.name.lower() == "notebooks" else cwd

# Garante que imports "from src...." funcionem
sys.path.insert(0, str(PROJECT_ROOT))

# Opcional: garante que arquivos relativos (data/...) resolvam a partir de optimization/
os.chdir(PROJECT_ROOT)

APP_BACKEND = (PROJECT_ROOT.parent / "rag-backend").resolve()
APP_BACKEND_DATA = APP_BACKEND / "data"
APP_BACKEND_STATIC = APP_BACKEND / "app" / "static"

print('Working dir (Local):', os.getcwd())
print("cwd:", cwd)
print("PROJECT_ROOT:", PROJECT_ROOT)
print("sys.path[0]:", sys.path[0])

In [2]:
import math
import random
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
from src.core.medical_genetic_algorithm import (
    carregar_veiculos_csv,
    carregar_entregas_csv,
    executar_ga_vrp,
    GAConfig,
)
from src.models.base_default import BASE_PADRAO
from src.models.models import Rota
from src.visualization.gerar_mapa_rotas_vrp import gerar_mapa_rotas_vrp

## 2. Regenerar `entregas.csv` e `rotas_iniciais.csv`


In [ ]:
%run src/utils/gerar_csv_entregas_rotas_iniciais.py


## 3. Inspecionar rapidamente os dados

Nesta etapa carregamos e conferimos:
- `data/hospitais_df.csv` – 100 hospitais e unidades de saúde, com latitude/longitude completas;
- `data/veiculos.csv` – parâmetros operacionais dos veículos;
- `data/entregas.csv` – entregas geradas a partir dos hospitais.


In [ ]:
# Hospitais (fonte original)
hospitais_path = PROJECT_ROOT / 'data' / 'hospitais_df.csv'
df_hospitais = pd.read_csv(hospitais_path)
print('Total de hospitais:', len(df_hospitais))
df_hospitais.head()


In [ ]:
# Veículos
veiculos_path = PROJECT_ROOT / 'data' / 'veiculos.csv'
df_veiculos = pd.read_csv(veiculos_path)
print('Total de veículos:', len(df_veiculos))
df_veiculos.head()


In [ ]:
# Entregas geradas a partir dos hospitais
entregas_path = PROJECT_ROOT / 'data' / 'entregas.csv'
df_entregas = pd.read_csv(entregas_path)
print('Total de entregas:', len(df_entregas))
df_entregas.head()


## 4. Otimizar Rotas

In [ ]:
from src.models.models import Entrega, Veiculo

base = BASE_PADRAO
entregas: list[Entrega] = carregar_entregas_csv(str(entregas_path))
veiculos: list[Veiculo] = carregar_veiculos_csv(str(veiculos_path))

print('Base logística:', base)
print('Qtde entregas:', len(entregas))
print('Qtde veículos:', len(veiculos))


### Configuração do Algoritmo Genético

A configuração abaixo reflete os parâmetros utilizados nos experimentos principais:

- `tamanho_populacao = 150`
- `geracoes = 1000`
- `taxa_mutacao = 0.25`
- `elitismo = 0.03`


In [ ]:
config = GAConfig(
    tamanho_populacao=150,
    geracoes=1000,
    taxa_mutacao=0.25,
    elitismo=0.03,
)
config


### Execução do GA VRP com população semeada

Aqui chamamos `executar_ga_vrp`, que:
- constrói a população inicial **semeada** a partir de `data/rotas_iniciais.csv`;
- evolui essa população ao longo das gerações;
- retorna as melhores rotas finais e o histórico do melhor fitness por geração.


In [9]:
rotas_resultado, historico_fitness = executar_ga_vrp(
    entregas=entregas,
    veiculos=veiculos,
    base=base,
    config=config,
    caminho_rotas_iniciais=str(PROJECT_ROOT / 'data' / 'rotas_iniciais.csv'),
)

In [ ]:
# ============================
# CSV – Detalhe das rotas (rota × entrega)
# ============================

dados_detalhados = []

for rota in rotas_resultado:
    veiculo_id = rota.veiculo.id_veiculo

    for ordem, entrega in enumerate(rota.entregas, start=1):
        dados_detalhados.append({
            'veiculo_id': veiculo_id,
            'ordem_na_rota': ordem,
            'entrega_id': entrega.id_entrega,
            'hospital_id': entrega.id_hospital,
            'hospital_nome': entrega.nome,
            'prioridade': entrega.prioridade.name,
            'peso_kg': entrega.peso_kg,
            'tempo_entrega_min': entrega.tempo_estimado_entrega_min,
            'lat': entrega.localizacao[0],
            'lng': entrega.localizacao[1],
        })

df_rotas_detalhado = pd.DataFrame(dados_detalhados)

csv_detalhe_path = PROJECT_ROOT / 'data' / 'resultados' / 'rotas_otimizadas.csv'
df_rotas_detalhado.to_csv(csv_detalhe_path, index=False)

#Exportar para o backend RAG
path_rag_backend = (APP_BACKEND_DATA / 'rotas_otimizadas.csv')
df_rotas_detalhado.to_csv(path_rag_backend, index=False)

print(f"CSV detalhado exportado para: {csv_detalhe_path}")
df_rotas_detalhado.head()


In [ ]:
# Encontrar geração com o melhor (menor) fitness
total_gen = len(historico_fitness)
melhor_gen_idx = min(range(len(historico_fitness)), key=lambda i: historico_fitness[i])
melhor_gen = melhor_gen_idx + 1  # ajusta para 1-based
melhor_valor = historico_fitness[melhor_gen_idx]
melhoria_absoluta = historico_fitness[0] - historico_fitness[-1]
melhoria_relativa_pct = (melhoria_absoluta / historico_fitness[0]) * 100 if historico_fitness[0] != 0 else 0.0

print(f"Total de gerações: {total_gen}")
print(f"Fitness inicial (geração 1)         : {historico_fitness[0]:.2f}")
print(f"Fitness final (geração {total_gen})        : {historico_fitness[-1]:.2f}")
print(f"Melhor fitness observado            : {melhor_valor:.2f}")
print(f"Geração do melhor fitness           : {melhor_gen}")
print(f"Melhoria absoluta (inicial - final) : {melhoria_absoluta:.2f}")
print(f"Melhoria relativa                   : {melhoria_relativa_pct:.2f}%")

### Evolução do fitness ao longo das gerações

O gráfico abaixo mostra a evolução do **melhor fitness** em cada geração. Lembrando que, neste problema, **quanto menor o fitness, melhor a solução**.


In [ ]:
# Curva normal de melhor fitness por geração
geracoes = list(range(1, len(historico_fitness) + 1))

plt.figure(figsize=(10, 5))
plt.plot(geracoes, historico_fitness, label="Melhor fitness da geração")

# Marca o ponto da melhor geração global
plt.scatter([melhor_gen], [melhor_valor], s=80, marker="o", label="Melhor fitness global")

# Linha vertical na melhor geração
plt.axvline(melhor_gen, linestyle="--", alpha=0.7)

plt.xlabel("Geração")
plt.ylabel("Melhor fitness (menor é melhor)")
plt.title("Evolução do melhor fitness por geração – VRP população semeada")
plt.grid(True)
plt.ylim(min(historico_fitness) * 0.9, historico_fitness[50] * 1.1)
plt.legend()
plt.show()


## 5. Resultados

In [ ]:
# ============================
# Rotas otimizadas – métricas
# ============================

dados_rotas_resultados = []

for rota in rotas_resultado:
    dados_rotas_resultados.append({
        'veiculo_id': rota.veiculo.id_veiculo,
        'distancia_km': rota.distancia_total_km,
        'tempo_min': rota.tempo_total_min,
        'custo_total': rota.custo_total(),
        'qtd_entregas': len(rota.entregas),
    })

df_resumo_resultados = pd.DataFrame(dados_rotas_resultados)

path_resumo_resultados = PROJECT_ROOT / 'data' / 'resumo_resultados.csv'
df_resumo_resultados.to_csv(path_resumo_resultados, index=False)

path_resumo_resultados_backend = APP_BACKEND_DATA / 'resumo_resultados.csv'
df_resumo_resultados.to_csv(path_resumo_resultados_backend, index=False)

print('Métricas das rotas otimizadas:')
print(df_resumo_resultados)

# Estatísticas gerais
print('\nEstatísticas gerais das rotas otimizadas:')
print('Distância total (km):', df_resumo_resultados['distancia_km'].sum())
print('Tempo total (min):', df_resumo_resultados['tempo_min'].sum())
print('Custo total (R$):', df_resumo_resultados['custo_total'].sum())

### Geração e visualização do mapa das rotas (Folium)
Por fim, geramos o mapa HTML com as rotas VRP utilizando a função `gerar_mapa_rotas_vrp`. Cada veículo aparece em uma camada própria, permitindo ativar/desativar a visualização de forma interativa.


In [ ]:
import sys
from IPython.display import display

# Caminho absoluto do arquivo de saída
mapa_path = PROJECT_ROOT / 'data' / 'resultados' / 'mapa_rotas_vrp.html'
path_rag_backend_mapa = APP_BACKEND_STATIC / 'mapa_rotas_vrp.html'
paths = [mapa_path, path_rag_backend_mapa]

# Gera o mapa e obtém o objeto folium.Map
mapa_final = gerar_mapa_rotas_vrp(
    rotas_resultado,
    lista_paths=paths,
    salvar_arquivo=True,
)

mapa_final

